# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
vac=pd.read_csv('weather.csv')
vac.head()

,Unnamed: 0,id,city_name,Humidity,Longitud,Latitud,Temperatura,Nubes,Velocidad
0,0,5282297,Bethel,91,-73.4140,41.3712,65.59,100,13.09
1,1,5122534,Jamestown,100,-79.2353,42.0970,44.58,90,17.27
2,2,1218436,Saýat,56,63.8803,38.7839,69.80,75,13.80
3,3,149812,Sokoni,100,39.5500,-6.4333,73.40,40,9.40
4,4,2155415,New Norfolk,47,147.0587,-42.7826,66.31,24,1.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

In [25]:
ciudades=vac[['Latitud','Longitud']]
humedad=vac['Humidity']
imagen=gmaps.figure()
capa=gmaps.heatmap_layer(ciudades)
imagen.add_layer(capa)
imagen

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
df3=vac[(vac['Temperatura']>=50)&(vac['Temperatura']<=70)]
df3=df3[df3['Velocidad']<=12]
df3=df3[df3['Nubes']<=0]
df3

,Unnamed: 0,id,city_name,Humidity,Longitud,Latitud,Temperatura,Nubes,Velocidad
30,30,3466165,Cidreira,78,-50.2056,-30.1811,64.62,0,7.18
42,42,107304,Buraidah,39,43.9750,26.3260,66.20,0,4.61
52,52,2164837,Goulburn,63,149.7167,-34.7500,66.96,0,5.12
88,88,964420,Port Elizabeth,88,25.5701,-33.9180,55.40,0,6.91
105,105,3442568,Fray Bentos,89,-58.2956,-33.1325,55.99,0,1.99
140,140,3443061,Chui,84,-53.4616,-33.6971,55.20,0,4.59
211,211,3247554,Čelinac,87,17.3278,44.7228,51.80,0,4.61
219,219,2176639,Batemans Bay,59,150.1833,-35.7167,69.33,0,8.68
232,232,102527,Sakakah,19,40.2064,29.9697,66.20,0,3.44
236,236,1792087,Yinzhu,32,119.9753,35.8786,64.40,0,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df=df3
hotel_df['Hotel Name']=""
hotel_df

,Unnamed: 0,id,city_name,Humidity,Longitud,Latitud,Temperatura,Nubes,Velocidad,Hotel Name
30,30,3466165,Cidreira,78,-50.2056,-30.1811,64.62,0,7.18,
42,42,107304,Buraidah,39,43.9750,26.3260,66.20,0,4.61,
52,52,2164837,Goulburn,63,149.7167,-34.7500,66.96,0,5.12,
88,88,964420,Port Elizabeth,88,25.5701,-33.9180,55.40,0,6.91,
105,105,3442568,Fray Bentos,89,-58.2956,-33.1325,55.99,0,1.99,
140,140,3443061,Chui,84,-53.4616,-33.6971,55.20,0,4.59,
211,211,3247554,Čelinac,87,17.3278,44.7228,51.80,0,4.61,
219,219,2176639,Batemans Bay,59,150.1833,-35.7167,69.33,0,8.68,
232,232,102527,Sakakah,19,40.2064,29.9697,66.20,0,3.44,
236,236,1792087,Yinzhu,32,119.9753,35.8786,64.40,0,4.47,


In [53]:
for i,j in hotel_df.iterrows():
    time.sleep(0.5)
    try:
        google='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        parametros={
            "keyword": "hotel",
            "radius": 5000,
            "key": g_key,
        }
        Latitud=j['Latitud']
        Longitud=j['Longitud']
        parametros['location']=f"{Latitud},{Longitud}"
        hotel_list=requests.get(google,params=parametros).json()
        hotel_df.loc[i,'Hotel Name']=hotel_list["results"][0]["name"]
    except IndexError:
        hotel_df.loc[i,'Hotel Name']="No data"
        
hotel_df.rename(columns={'Latitud':'Lat', 'Longitug':'Lng', 'city_name':'City'})
hotel_info=hotel_df

In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
